# Bayesian MCA (Metabolic Control Analysis)

## Load packages

In [1]:
%load_ext nb_black

import cobra, gzip, re

import numpy as np
import pandas as pd
import sys

sys.path.append("../")
from src.eflux2 import EFlux2

<IPython.core.display.Javascript object>

## 1. Provide Required Input Content

### 1.1. Required content filenames

In [4]:
config = {}

<IPython.core.display.Javascript object>

In [5]:
# Model (Needs to be [verified for] flipped, non-zero with respect to reference strain)
config["model_fname"] = '../models/iJB1325_HP.nonnative_genes.pubchem.flipped.nonzero.reduced.json'
config["model_solver"] = 'glpk'#'gurobi'
# !!!! Issues with 'gurobi' solver
config["model_objective"] = 'BOUNDARY_He'

# List of Strains
config["line_strain_list_fname"] = '../data/experiment_w_reps.tsv'

# Reference Strain
config["ref_line"] = 'SF ABF93_7-R3'


<IPython.core.display.Javascript object>

### 1.2. Raw Data

In [7]:
### Protein Concentrations (proteomics)

# Protein to gene map
config["protein_to_gene_map_fname"] = "../data/Aspni7_FilteredModels1_deflines.gff3.gz"

### Uniprot not needed....
# Uniprot: list of proteins in A.niger, including non-natives
# config["uniprot_fname"] = '../data/Uniprot_ANiger.csv'


### EDD info
config["study_slug"] = "aniger_3hp_omics_shakeflask-93_pnnl"
config["edd_server"] = "edd.agilebiofoundry.org"
config["user"] = "zucker"

# Internal Metabolite Concentrations (metabolomics)
# met_conc_fname = '../data/metabolite_concentrations.csv'
# (x_file)
# !!! THIS can come from EDD + mapping IDs

# External Metabolite Time Series
# ext_met_ts_fname = '../data/normalized_external_metabolites.csv'
# !!! this isn't a time series...
# (y_file)
# # !!! THIS can come from EDD + mapping IDs


# Line Rates
config["line_rates_fname"] = "../data/a.niger_strain_specific_rates.csv"


# Protocol to Omics-data type mapping
config["protocol_to_omics_map"] = {
    "Flux_Constraint": "PNNL Global Prot Intensity",
    "Enzyme_Abundance": "Targeted Proteomics",
    "Internal_Metabolomics": "",
    "External_Metabolomics": "",
    "Internal_Fluxes": "",
    "External_Fluxes": "",
}

config["protocol_to_omics_map"]["Enzyme_Abundance"]


# ===> v_star_file = '../data/Eflux2_flux_rates.flipped.csv'
# ===> v_file = '../data/Eflux2_flux_rates.flipped.csv'
# ===> e_file = '../data/normalized_targeted_enzyme_activities.csv'

'Targeted Proteomics'

<IPython.core.display.Javascript object>

### 1.3. Mapping files
NOTE: Will likely be data specific; may be developed by hand

In [8]:
# maps protein IDs to model gene IDs
proteinID_to_modelGene_map = {}

# maps metabolite IDs to model metabolite IDs
metaboliteID_to_modelMetabolite_map = {}

# maps data file strain IDs to canonical strain IDs (wrt EDD)
dataFileStrainID_to_canonicalStrainID_map = {}

<IPython.core.display.Javascript object>

## 2. Pre-processing

### 2.1. Extract and organize content from provided filenames

#### 2.1.1. Model

In [11]:
### Model
# Get model from file
model = cobra.io.load_json_model(config["model_fname"])

# Set model attributes
model.solver = config["model_solver"]
model.objective = config["model_objective"]

<IPython.core.display.Javascript object>

#### 2.1.2. Line/Strain info

In [12]:
### Lines list

# Get lines list from file
line_descr_list_df = pd.read_csv(config["line_strain_list_fname"], sep="\t")

# Map line descriptions with ICE IDs
line_descr_list_df["ICE"] = line_descr_list_df["Strain"].str.replace(
    "https://registry.agilebiofoundry.org/entry/", "ABF_00"
)
line_descr_list_df = (
    line_descr_list_df.sort_values(["ICE", "Line Name"]).dropna().set_index("Line Name")
)

# Get line names
line_list = line_descr_list_df.index

<IPython.core.display.Javascript object>

In [14]:
### Line rates
ref_line = config["ref_line"]

# Get line rates from file
line_rates = pd.read_csv(config["line_rates_fname"])

# Map line rates with ICE IDs
line_rates = line_rates.sort_values(["Strain (ICE)", "Omics Sample ID"]).dropna()
if all(line_rates["Strain (ICE)"].values == line_descr_list_df["ICE"].values):
    line_rates.index = line_descr_list_df.index
else:
    print("Error: ICE IDs dont match between line_strain_list and line_rates files")
# line_rates.to_csv('line_rates.csv')

# Normalize line uptake and secretion rates wirth respect to reference line glucose uptake rate
ref_line_glucose_rate = line_rates.loc[
    ref_line, "glucose_uptake_rates (mmol/gDCW * hr)"
]
normalized_line_rates = line_rates.drop(
    ["Omics Sample ID", "Strain (ICE)", "Genotype"], axis=1
)
normalized_line_rates = normalized_line_rates.divide(ref_line_glucose_rate, axis=1)


# Get strain rates (is this needed?)
# strain_rates = line_rates.groupby(['Strain (ICE)', 'Genotype']).mean()
# strain_rates_std = line_rates.groupby(['Strain (ICE)', 'Genotype']).std()
# strain_rates.to_csv('Strain_rates_mean.csv')
# strain_rates_std.to_csv('Strain_rates_std.csv')

# Get reference line's strain ID (is this needed?)
# ref_strain = strain_list_df['ICE'][strain_list_df.index == ref_line][0]

<IPython.core.display.Javascript object>

#### 2.1.3. Download and extract data from EDD

In [ ]:
### Download the data from EDD
session = login(edd_server=config["edd_server"], user=config["user"])
EDD_df = export_study(session, config["study_slug"], edd_server=config["edd_server"])
EDD_df.groupby(["Line ID", "Line Name", "Line Description"]).count()

In [23]:
### Read data already downloaded from EDD
EDD_df = pd.read_csv("../data/EDD.csv.gz")
EDD_df.head()

,Unnamed: 0,Study ID,Study Name,Line ID,Line Name,Line Description,Protocol,Assay ID,Assay Name,Formal Type,Measurement Type,Compartment,Units,Value,Hours
0,0,45008,A.niger_3HP_Omics_Shakeflask 93_PNNL,45009,SF ABF93_1-R1,ATCC11414 Parent,HPLC,45063,SF ABF93_1-R1,cid:5793,"(3R,4S,5S,6R)-6-methyloltetrahydropyran-2,3,4,...",0,Percent,81.62472,72.0
1,1,45008,A.niger_3HP_Omics_Shakeflask 93_PNNL,45009,SF ABF93_1-R1,ATCC11414 Parent,HPLC,45063,SF ABF93_1-R1,cid:5793,"(3R,4S,5S,6R)-6-methyloltetrahydropyran-2,3,4,...",0,Percent,56.79776,120.0
2,2,45008,A.niger_3HP_Omics_Shakeflask 93_PNNL,45009,SF ABF93_1-R1,ATCC11414 Parent,HPLC,45063,SF ABF93_1-R1,cid:5793,"(3R,4S,5S,6R)-6-methyloltetrahydropyran-2,3,4,...",0,Percent,65.36000,168.0
3,3,45008,A.niger_3HP_Omics_Shakeflask 93_PNNL,45010,SF ABF93_1-R2,ATCC11414 Parent,HPLC,45064,SF ABF93_1-R2,cid:5793,"(3R,4S,5S,6R)-6-methyloltetrahydropyran-2,3,4,...",0,Percent,79.45385,72.0
4,4,45008,A.niger_3HP_Omics_Shakeflask 93_PNNL,45010,SF ABF93_1-R2,ATCC11414 Parent,HPLC,45064,SF ABF93_1-R2,cid:5793,"(3R,4S,5S,6R)-6-methyloltetrahydropyran-2,3,4,...",0,Percent,43.84099,120.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298856,298856,45008,A.niger_3HP_Omics_Shakeflask 93_PNNL,45056,SF ABF93_16-R3,an4104(PYC+3OP2D),PNNL Global Prot Intensity,46341,SF ABF93_16-R3,G3XM75,NaN,0,intensity,31.52222,120.0
298857,298857,45008,A.niger_3HP_Omics_Shakeflask 93_PNNL,45056,SF ABF93_16-R3,an4104(PYC+3OP2D),PNNL Global Prot Intensity,46341,SF ABF93_16-R3,G3Y8S4,NaN,0,intensity,33.15461,120.0
298858,298858,45008,A.niger_3HP_Omics_Shakeflask 93_PNNL,45056,SF ABF93_16-R3,an4104(PYC+3OP2D),PNNL Global Prot Intensity,46341,SF ABF93_16-R3,G3Y7D7,NaN,0,intensity,32.76605,120.0
298859,298859,45008,A.niger_3HP_Omics_Shakeflask 93_PNNL,45056,SF ABF93_16-R3,an4104(PYC+3OP2D),PNNL Global Prot Intensity,46341,SF ABF93_16-R3,G3Y4S3,NaN,0,intensity,25.50885,120.0


<IPython.core.display.Javascript object>

### 2.2. Pre-Process Protein Concentrations

#### 2.2.1. Maps from file

In [15]:
# Protein to gene map (& gene to protein)
prot_gene_map, gene_prot_map = {}, {}
prot_gene_re = re.compile(r"proteinId=(\d+);.*transcriptId=(\d+)")
with gzip.open(config["protein_to_gene_map_fname"], "rt") as gff3:
    for line in gff3:
        m = prot_gene_re.search(line)
        if m:
            gene_prot_map[m.group(2)] = m.group(1)
            prot_gene_map[m.group(1)] = m.group(2)

<IPython.core.display.Javascript object>

# Map uniprot to protein and transcript ID
uniprot = (
    pd.read_csv(uniprot_fname, index_col="Entry name")["ID"]
    .str.split("|")
    .str.get(-1)
    .to_frame("ID")
)
uniprot["Transcript"] = uniprot["ID"].replace(
    to_replace=prot_gene_map, value=None
)

#### 2.2.2. Function for mapping to reactions to experiment

In [16]:
# Define function to get enzyme activity as a function of protein expression
def get_enzyme_activity_expression(proteins, model):
    """Get enzyme activity expression as a function of protein expression

    Take the min over all subunits for each isoenzyme and sum over all isoenzymes

    :param proteins:  protein x experiment dataframe
    :param model:  cobra model
    :returns:  reaction x experiment dataframe
    """
    enzyme_expression = {}
    transcripts = list(proteins.index)
    for r in model.reactions:
        if r.gene_reaction_rule and (
            len(set([g.id for g in r.genes]) & set(transcripts)) > 0
        ):
            subunits_expression = {}
            for x in [x.strip("() ") for x in r.gene_reaction_rule.split(" or ")]:
                # Take the min over all subunits for each study line
                subunits = [
                    y.strip("() ")
                    for y in x.split(" and ")
                    if y.strip("() ") in transcripts
                ]
                if len(subunits) > 0:
                    subunits_expression[x] = proteins.loc[subunits].min(axis=0)
            enzyme_expression[r.id] = pd.DataFrame(subunits_expression).sum(axis=1)
    enzyme_expression = pd.DataFrame(enzyme_expression).T
    enzyme_expression.index.name = "rxn"
    return enzyme_expression

<IPython.core.display.Javascript object>

#### 2.2.3. Proteomics for Flux Constraints

In [20]:
pd.Series(prot_gene_map).to_frame("Transcript")

,Transcript
35362,35362
35369,35369
35382,35382
35417,35417
35418,35418
...,...
46163,46163
1098883,1099159
1098889,1099165
1098890,1099166


<IPython.core.display.Javascript object>

In [21]:
### TODO change variable name from global_proteomics to flux_constraints

# Extract global proteomics from EDD data
flux_constraints = EDD_df[
    EDD_df["Protocol"] == config["protocol_to_omics_map"]["Flux_Constraint"]
]

# Normalize global proteomics with ??uniprot??

normalized_flux_constraints = global_proteomics[
    ["Formal Type", "Line Name", "Value"]
].pivot_table(index="Formal Type", columns="Line Name", values="Value")

normalized_flux_constraints = (
    normalized_flux_constraints.divide(normalized_flux_constraints[ref_line], axis=0)
    .replace(-np.inf, 1)
    .replace(np.inf, 1)
    .fillna(1)
)

# Apply JGI names (model genes) to Normalize global proteomics
normalized_flux_constraints_jgi = (
    normalized_flux_constraints_jgi.join(
        pd.Series(prot_gene_map).to_frame("Transcript")
    )
    .set_index("Transcript")
)
#normalized_flux_constraints_jgi = normalized_flux_constraints_jgi.drop("ID", axis=1)

# Map global proteomics
# Map enzyme abundances to model gene IDs
get_enzyme_activity_expression(normalized_flux_constraints_jgi, model)

NameError: name 'EDD_df' is not defined

<IPython.core.display.Javascript object>

#### 2.2.4. Proteomics for Enzyme Abundance

In [2]:
### TODO change variable name from targeted_proteomics to enzyme_abundance


# Extract targeted proteomics from EDD data
enzyme_abundance = EDD_df[
    EDD_df["Protocol"] == config["protocol_to_omics_map"]["Enzyme_Abundance"]
]

# Normalize targeted proteomics with ??uniprot??
normalized_enzyme_abundance = enzyme_abundance.set_index(
    ["Formal Type", "Line Name"]
)["Value"].unstack(fill_value=1)
normalized_enzyme_abundance = normalized_enzyme_abundance.divide(
    normalized_enzyme_abundance[ref_line], axis=0
)

# JGI normalize targeted proteomics
normalized_enzyme_abundance_jgi = (
    normalized_enzyme_abundance.join(
        pd.Series(prot_gene_map).to_frame("Transcript")
    )
    .dropna()
    .set_index("Transcript")
)

# Map targeted proteomics
normalized_rxns_targeted_prot = get_enzyme_activity_expression(
    normalized_enzyme_abundance_jgi, model
)

NameError: name 'EDD_df' is not defined

<IPython.core.display.Javascript object>

### 2.3. Pre-Process Metabolite Concentrations

#### 2.3.1. Maps for Metabolomics

In [ ]:
# For mapping pubchem to model IDs
metab = pd.read_excel(
    "Multiomics/210525_Aspergillus_niger_3HP_ABFSF93_multiomics_data_fixed.xlsx",
    sheet_name="Intra_metabolites_NOTnormalized",
    engine="openpyxl",
)


pubchem_metabs = metab[~metab["PubChem"].isnull()].set_index("PubChem")

kegg_metabs = metab[~metab["Kegg"].isnull()].set_index("Kegg")

from collections import defaultdict

def make_metabolite_map(db, model, prefix=None):
    db_model_mets = defaultdict(list)
    for m in model.metabolites:
        if db in m.annotation:
            if prefix:
                db_model_mets[f"{prefix}:{m.annotation[db]}"].append(m)
            else:
                db_model_mets[m.annotation[db]].append(m)
    return db_model_mets


kegg_model_mets = make_metabolite_map("kegg.compound", model)
len(set(kegg_metabs.index) & set(kegg_model_mets.keys()))




### --------------------------------------------------------------------------------



from copy import deepcopy

def add_db_annotations_to_model(
    src_model,
    annot_db_name,
    data_db_name,
    model_metabolite_map,
    data_metabolite_map,
    compartment_restrictions=["c"],
):
    model = deepcopy(src_model)
    for db_id, metabolites in model_metabolite_map.items():
        if db_id in data_metabolite_map.index:
            print(db_id, ",".join(m.id for m in metabolites))
            for m in metabolites:
                if m.compartment in compartment_restrictions:
                    m.annotation[annot_db_name] = data_metabolite_map.loc[
                        db_id, data_db_name
                    ]
    return model

model_w_pubchem = add_db_annotations_to_model(
    model, "pubchem", "PubChem", kegg_model_mets, kegg_metabs, ["c"]
)

pubchem_model_map = dict([
    (f"""cid:{m.annotation['pubchem']}""", m.id)
    for m in model_w_pubchem.metabolites
    if ("pubchem" in m.annotation) and ("c" == m.compartment)
    ])

def map_pubchem_to_model_id(pubchem_model_map, dataset):
    idx = [i for i in dataset.index if i in pubchem_model_map]
    return dataset.loc[idx].rename(index=pubchem_model_map)


#### 2.3.2. Internal Metabolites

In [25]:
# Extract internal metabolites from EDD data
internal_metab = EDD_df[EDD_df["Protocol"] == "PNNL Global Metabolomics (intracellular)"]


# Normalize internal metabolites
normalized_internal_metab = (
    internal_metab.set_index(["Measurement Type", "Formal Type", "Line Name"])["Value"]
    .unstack()
    .apply(np.log2)
)
normalized_internal_metab = (
    normalized_internal_metab.sub(normalized_internal_metab[reference_strain], axis=0)
    .replace(-np.inf, 0)
    .fillna(0)
    .replace(np.inf, 0)
)

# Add model IDs to data
pubchem_ids = list(
    internal_metab.groupby(["Formal Type", "Measurement Type"]).count().index
)
normalized_internal_metab_w_model_ids = map_pubchem_to_model_id(
    pubchem_model_map, normalized_internal_metab.droplevel(0)
)

,ID,Transcript
Entry name,,
A7U8C7,PAND_Tribolium_castaneum,PAND_Tribolium_castaneum
C2ZAL1,BAPAT_Bacillus_cereus,BAPAT_Bacillus_cereus
P3983,HPDH_escherichia_coli,HPDH_escherichia_coli
ABF_005934,PAND_Tribolium_castaneum,PAND_Tribolium_castaneum
ABF_005935,BAPAT_Bacillus_cereus,BAPAT_Bacillus_cereus
...,...,...
G3XM75,56950,56950
G3Y8S4,57046,57046
G3Y7D7,57150,57150


In [ ]:
### Fluxes

## !!! Hard coded for 3hp

# Run Eflux2 to calculate fluxes
fluxes = {}
for rep in global_prot.columns:
    with model:
        glucose_uptake = normalized_line_rates.loc[rep,'glucose_uptake_rates (mmol/gDCW * hr)' ]
        secrete_3hp = normalized_line_rates.loc[rep,'3hp_secretion_rates (mmol/gDCW * hr)']
        if glucose_uptake:
            model.reactions.BOUNDARY_GLCe.upper_bound = glucose_uptake
        if secrete_3hp:
            model.reactions.EX_3hpp_e.lower_bound =  secrete_3hp
        try:
            print(rep)
            fluxes[rep] = EFlux2(model, global_prot[rep])
        except TypeError:
            print(f"Replicate {rep} with glucose {glucose_uptake} and 3hp {secrete_3hp} is infeasible")
fluxes[rep]

efluxes = pd.DataFrame(dict([(rep, fluxes[rep].to_frame()['fluxes']) for rep in fluxes]))
reduced_costs = pd.DataFrame(dict([(rep, fluxes[rep].to_frame()['reduced_costs']) for rep in fluxes]))
eflux_rates = efluxes*ref_line_glucose_rate

#efluxes.to_csv('Eflux2_flux_yield.csv')
#reduced_costs.to_csv('Eflux2_yield_reduced_costs.csv')
#eflux_rates.to_csv('Learn/Eflux2_flux_rates.csv')

In [20]:
### Internal(?) metabolites

# Get metabolite concentrations from file
met_conc = pd.read_csv(met_conc_fname, index_col=0)
met_conc = met_conc.loc[[m.id for m in model.metabolites if m.id in met_conc.index]]

In [21]:
### External metabolites

# Get external metabolites from file
ext_met_ts = pd.read_csv(ext_met_ts_fname, index_col=0)
ext_met_ts = ext_met_ts.loc[[m.id for m in model.metabolites if m.id in ext_met_ts.index]]

,SF ABF93_1-R1,SF ABF93_1-R2,SF ABF93_1-R3,SF ABF93_10-R1,SF ABF93_10-R2,SF ABF93_10-R3,SF ABF93_11-R1,SF ABF93_11-R2,SF ABF93_11-R3,SF ABF93_12-R1,...,SF ABF93_6-R3,SF ABF93_7-R1,SF ABF93_7-R2,SF ABF93_7-R3,SF ABF93_8-R1,SF ABF93_8-R2,SF ABF93_8-R3,SF ABF93_9-R1,SF ABF93_9-R2,SF ABF93_9-R3
bDGLCe,0.96856,0.595006,1.019095,0.306643,0.370181,0.332601,0.286244,0.425582,0.483575,0.327102,...,0.236882,0.061311,0.077775,0.0,0.636998,0.488817,0.733679,0.646963,0.425260,0.528886
3hpp_e,0.00000,0.000000,0.000000,-0.032113,-0.071955,-0.125552,-0.478471,-0.538206,-0.463320,-0.012481,...,-0.453518,0.109321,0.070619,0.0,-1.247424,-1.177829,-1.291246,-0.445347,-0.466815,-0.611556


### 2.2. Check input file/data formats

In [ ]:
# Ensure all fluxes are positive with respect to reference strain


# Ensure all data have same line IDs


# Ensure columns line up across data sets; error message if not


# Ensure all rows correspond to model variables across data sets


### 2.3. Calculate rates

In [2]:
# Growth Rates

# Uptake Rates

# Secretion Rates


### 2.4. Map data identifiers

In [ ]:
# Map metabolomics data

# Map proteomics data

# Automated mapping: use InChi mapping where available

# Semi-Manual:
#     Metabolites: PubChem, Metacyc (metabolite translation service), MetaNetX, PNNL stuff
#     Proteins: BBH (bidirectional best hits)


### 2.5. Normalize metabolomics & proteomics data wrt reference strain ==> reference strain = 1

### 2.6. Generate Enzyme Data

Input: Proteomics data, model, protein to enzyme-reaction map

Output: enzyme to reaction map; 

Example: (Gene A and Gene B) or Gene C --> Enzyme Activity X

Helper Function: use model to obtain protein to enzyme-reaction map (see "get_enzyme_activity_expression()" A.niger_MultiOmics Section 4.1)

### 2.6. Generate eFlux Data

Input: Proteomics data, model, growth rates, uptake rates, secretion rates

Output: strain specific set of fluxes (different than enzyme activities; same shape but for reaction rate)

Helper function: see [https://github.com/AgileBioFoundry/AspergillusQ4Milestone/blob/main/notebooks/Eflux4A.niger.ipynb] section "Normalize uptake and secretion rates by glucose uptake rate of reference study line." where "fluxes" are defined
    

## 3. Main Process

In [ ]:
# Plot heatmaps (see A.niger_MultiOmics)

In [ ]:
# Normalized Enzyme Activites
normalized_enzyme_activities_fname = '../data/normalized_targeted_enzyme_activities.csv'




### Define wild-type strain IDs
wild_type = ['SF ABF93_1-R1', 'SF ABF93_1-R2','SF ABF93_1-R3']

In [1]:
# Load Model
model = cobra.io.load_json_model(model_fname)

# Load normalized enzyme activites as a dataframe
normalized_enzyme_activities_df = pd.read_csv(normalized_enzyme_activities_fname,index_col=0)

In [ ]:
# Remove wild-type from normalized enzyme activities dataframe
